In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import zipfile
import os
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
from torch.optim.lr_scheduler import StepLR


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gdown
!pip install zipfile
!pip install fvcore
!pip install tqdm
!pip install pytorchvideo

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=d1c5f8660941060409aebe3c6b881a43ad78eafc34d1b2b285dd5dc33b00f525
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=9fbb4353ca6ac99cf4366e43e666b7130e15e0a262c364bda1c248b5f3b32571
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/13

In [ ]:
import gdown

# URL of your Google Drive file
#url = 'https://drive.google.com/uc?export=download&id=YOUR_FILE_ID'
#url = 'https://drive.google.com/uc?export=download&id=1pW5niXoC5VAO0x8M5G48F24Gp7Ji0cyZ'
#output = 'Face_Frames2.zip'  # Name of the file to be saved locally

#gdown.download(url, output, quiet=False)

In [ ]:
import zipfile
# Define the zip file path and the destination folder
zip_file_path = '/content/drive/MyDrive/Face_Frames2.zip'
destination_folder = '/content/'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print("Unzipping completed.")

Unzipping completed.


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
deceptive = datasets.ImageFolder(root='/content/Face_Frames2/Deceptive', transform=transform)
truthful = datasets.ImageFolder(root='/content/Face_Frames2/Truthful', transform=transform)

In [ ]:
deceptive[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
# Assuming deceptive is your dataset
deceptive_length = len(deceptive)
truthful_length = len(truthful)
num_samples = 30

# List to hold the new dataset
deceptive_data = []
truthful_data = []

for i in range(0, deceptive_length, num_samples):
    # Get the current sequence of 30 samples
    combined_samples = []
    for j in range(num_samples):
        if i + j < truthful_length:
            data, _ = deceptive[i + j]  # Extract the data part (ignore the label if present)
            combined_samples.append(data)

    # Stack the current sequence of samples along a new dimension
    if len(combined_samples) == num_samples:  # Ensure we have exactly 30 samples
        combined_samples = [combined_samples[0]] + combined_samples + [combined_samples[-1]]
        combined_tensor = torch.stack(combined_samples, dim=0)
        deceptive_data.append(combined_tensor)

for i in range(0, truthful_length, num_samples):
    # Get the current sequence of 30 samples
    combined_samples = []
    for j in range(num_samples):
        if i + j < truthful_length:
            data, _ = truthful[i + j]  # Extract the data part (ignore the label if present)
            combined_samples.append(data)

    # Stack the current sequence of samples along a new dimension
    if len(combined_samples) == num_samples:  # Ensure we have exactly 30 samples
        combined_samples = [combined_samples[0]] + combined_samples + [combined_samples[-1]]
        combined_tensor = torch.stack(combined_samples, dim=0)
        truthful_data.append(combined_tensor)

truthful_len = len(truthful_data)
deceptive_len = len(deceptive_data)

In [ ]:
truthful_data[0].shape
truthful_len
deceptive_data[0].shape
deceptive_len


825

In [ ]:
total_data = truthful_data + deceptive_data

In [ ]:
# Change 1st and 2nd dimensions of each sample
new_dataset = [sample.permute(1, 0, 2, 3) for sample in total_data]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_list, labels, transform=None):
        """
        Args:
            data_list (list): List of tensors, each of shape [30, 3, 224, 224].
            labels (list): List of labels corresponding to each tensor.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_list = data_list
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data = self.data_list[idx]
        label = self.labels[idx]

        if self.transform:
            data = self.transform(data)

        return data, label


# Assuming labels is a list of labels corresponding to each sample
labels = [0] * truthful_len + [1] * deceptive_len


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
# Create an instance of the custom dataset with labels and transform
#train_dataset = CustomDataset(new_dataset, labels, transform=None)
#train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)


# Save the dataset
#torch.save(train_dataset, 'train_dataset.pth')

In [ ]:
# Train-Test Split using sklearn
X = new_dataset
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

test_dataset = CustomDataset(X_test, y_test, transform=None)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

train_dataset = CustomDataset(X_train, y_train, transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)
model.blocks[-1].output_pool = nn.Identity()
model.blocks[-1].proj = nn.Linear(2048, 2, bias=True)
for i in range(len(model.blocks) - 2):
    model.blocks[i].requires_grad = False

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
def metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)

    return dict(
        accuracy=accuracy)


def train_epoch(model, optimizer, criterion, dataloader, device):
    model.train()
    losses = list()
    y_true_list = list()
    y_pred_list = list()
    batches = list()
    with tqdm(enumerate(dataloader), total=len(dataloader), leave=True) as iterator:
        for idx, batch in iterator:
            optimizer.zero_grad()
            x, y_true= batch
            prediction = model.forward(x.to(device))
            loss = criterion(prediction, y_true.to(device))
            loss.backward()
            optimizer.step()
            iterator.set_description(f"train loss={loss:.2f}")
            losses.append(loss)
            #y_true_list.append(y_true)
            #y_pred_list.append(prediction.argmax(-1))
            batches.append(idx)
    return torch.stack(losses), torch.cat(y_true_list), torch.cat(y_pred_list)



def test_epoch(model, criterion, dataloader, device):
    model.eval()
    with torch.no_grad():
        losses = list()
        y_true_list = list()
        y_pred_list = list()
        field_ids_list = list()
        batches = list()
        with tqdm(enumerate(dataloader), total=len(dataloader), leave=True) as iterator:
            for idx, batch in iterator:
                x, y_true = batch
                logits = model.forward(x.to(device))
                loss = criterion(logits, y_true.to(device))
                iterator.set_description(f"test loss={loss:.2f}")
                losses.append(loss)
                #y_true_list.append(y_true)
                #y_pred_list.append(logits.argmax(-1))
                batches.append(idx)
        return  torch.stack(losses), torch.cat(y_true_list), torch.cat(y_pred_list)

In [ ]:
#device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), 0.0001, weight_decay=0.01)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
log = list()
best_loss = float('inf')
patience = 3
n_epoch = 1
for epoch in range(n_epoch):
    train_loss, train_true, train_pred = train_epoch(model, optimizer, criterion, train_dataloader, device)
    test_loss, test_true, test_pred = test_epoch(model, criterion, test_dataloader, device)
    scores1 = metrics(test_true.cpu(), test_pred.cpu())
    scores= metrics(train_true.cpu(), train_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores.items()])
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy().mean()
    train_loss = train_loss.cpu().detach().numpy().mean()
    #scheduler.step()

    #print(f"epoch {epoch}: trainloss {train_loss:.2f}" + scores_msg)
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    log.append(scores1)

    # Early stopping check
    if test_loss < best_loss:
        best_loss = test_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break


    log_df = pd.DataFrame(log).set_index("epoch")
    #log_df.to_csv("Adam_lr0001-WDe-2.csv")

In [ ]:
name = '/content/drive/MyDrive/Models/Adam_lr00001Schedular-WDe-2.pth'
torch.save(model, name)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
"""# Load the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_deneme = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=False)
#model_deneme.load_state_dict(torch.load('lr0001-WDe-3_nosplit.pth', map_location=device))
model_deneme = torch.load("/content/drive/MyDrive/lr0001-WDe-3_nosplit.pth", map_location=device)
model_deneme.to(device)
model_deneme.eval()"""
device = 'cuda'
#model = torch.load('/content/drive/MyDrive/Adam_lr0001-WDe-2.pth', map_location=torch.device('cpu'))
model.to(device)
model.eval()
# Get predictions
y_pred = []
y_true = []
for batch, (X, y) in enumerate(test_dataloader):
    y_true.append(y)
    X = X.to(device)
    y = y.to(device)
    y_pred.append(model(X).argmax(-1).cpu().numpy())

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
accuracy_score(y_true, y_pred)



0.9446064139941691